In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torchvision import datasets
import torchvision.transforms as transforms
from tensorboardX import SummaryWriter


In [14]:
data_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Resize(32),
        transforms.Normalize((0.5),(1.0))
    ]
)

train_data = datasets.MNIST(root='./',train=True,download=True,transform=data_transform)
test_data = datasets.MNIST(root='./',train=False,download=True,transform=data_transform)

In [15]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32)

In [16]:
data, label = next(iter(train_loader))
print(data.shape)

torch.Size([32, 1, 32, 32])


In [20]:
# pytorch의 이미지 만드는 모델
class Lenet(nn.Module):
    def __init__(self): # 가장 먼저 실행되는 함수
        super(Lenet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6,kernel_size=5,stride=1) # Convolution Layer
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16,kernel_size=5,stride=1)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=120,kernel_size=5,stride=1)
        # Full connection 은 nn.Linear 로 생각 하면 된다
        self.fc1 = nn.Linear(in_features=120, out_features=84)
        self.fc2 = nn.Linear(in_features=84, out_features=10)

    def forward(self, x): # x 는 이미지
        x = self.conv1(x) # Convolution Layer 집어넣으니 1채널이 6채널이 됐다 (항상 매트릭스 형태를 띄움)
        x = F.tanh(x) # 활성화함수 집어넣고 -> 6, 28, 28
        x = F.max_pool2d(x,2,2) # max_pool2d -> 6, 14, 14

        x = self.conv2(x) 
        x = F.tanh(x) 
        x = F.max_pool2d(x,2,2) 

        x = self.conv3(x) 
        x = F.tanh(x)
        x = x.view(-1, 120) # 매트릭스를 1차원으로 펴기

        x = self.fc1(x)
        x = F.tanh(x) # 84

        x = self.fc2(x)
        x = F.tanh(x) # 10

        return x

model = Lenet()
model


Lenet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=120, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=10, bias=True)
)

In [21]:
from torchsummary import summary

summary(model,input_szie=(1,32,32))

Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            156
├─Conv2d: 1-2                            2,416
├─Conv2d: 1-3                            48,120
├─Linear: 1-4                            10,164
├─Linear: 1-5                            850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            156
├─Conv2d: 1-2                            2,416
├─Conv2d: 1-3                            48,120
├─Linear: 1-4                            10,164
├─Linear: 1-5                            850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0

In [22]:
lr = 1e-3
optim = Adam(model.parameters(), lr=lr)
epochs = 10
criterion = nn.CrossEntropyLoss()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

writer = SummaryWriter()

step = 0
for epoch in range(epochs):
    for data,label in train_loader:
        optim.zero_grad()
        pred = model(data.to(device))  #[32,1,32,32]
        loss = criterion(pred,label.to(device))
        writer.add_scalar('Loss/train',loss,step)
        step += 1

        loss.backward()
        optim.step()

    print(f'{epoch+1} loss : {loss.item()}')


1 loss : 0.8550175428390503
2 loss : 0.8170366287231445
3 loss : 0.8354312777519226
4 loss : 0.8667051792144775
5 loss : 0.8085789680480957
6 loss : 0.8307420015335083
7 loss : 0.8051958084106445
8 loss : 0.880725622177124
9 loss : 0.7966451048851013
10 loss : 0.796942949295044


In [ ]:
# 1. 모델 평가코드 작성
# 2. png이미지를 불러온다.
# 3. 이미지를 학습할때와 같이 전처리 한다.
# 4. 모델에 넣고 추론한다.
# 답: [-1, -1, -1, -1, 1, 34, , , , ,] 이런식으로 답이 나온다.